In [1]:
from tkinter import *
from tkinter import ttk
import tkinter.font as tkFont
from tkinter import font

In [2]:
from movie import Movie

In [3]:
from parse import parse_imdb_data

In [4]:
import os
import pickle

In [5]:
if os.path.isfile("movies.p"):
    movies = pickle.load(open("movies.p", "rb"))
else:
    movies = parse_imdb_data()
    pickle.dump(movies, open( "movies.p", "wb" ))

In [6]:
m_ = movies
#m_=list(filter(lambda m: m.year>2000, m_))
m_=list(filter(lambda m: m.num_ratings>1000, m_))
#m_=list(filter(lambda m: m.mean_rating>8.5, m_))
len(m_)

30587

In [7]:
def sortby(tree, col, descending):
    """sort tree contents when a column header is clicked on"""
    # grab values to sort
    data = [(tree.set(child, col), child) \
        for child in tree.get_children('')]
    # if the data to be sorted is numeric change to float
    #data =  change_numeric(data)
    # now sort the data in place
    if(data[0][0].isnumeric()): # FIX: doesn't work for float
        data = [(float(d[0]), d[1]) for d in data]
    data.sort(reverse=descending)
    for ix, item in enumerate(data):
        tree.move(item[1], '', ix)
    # switch the heading so it will sort in the opposite direction
    tree.heading(col, command=lambda col=col: sortby(tree, col, \
        int(not descending)))

In [8]:
min_year = min_rating_count = min_mean_rating = float('inf')
max_year = max_rating_count = max_mean_rating = float('-inf')
for m in m_:
    min_year = min((min_year, m.year))
    max_year = max((max_year, m.year))
    min_rating_count = min((min_rating_count, m.num_ratings))
    max_rating_count = max((max_rating_count, m.num_ratings))
    min_mean_rating = min((min_mean_rating, m.mean_rating))
    max_mean_rating = max((max_mean_rating, m.mean_rating))    

In [9]:
if os.path.isfile("ratings.p"):
    ratings = pickle.load(open("ratings.p", "rb"))
else:
    ratings = {}

In [10]:
ratings = {}

In [22]:
root = Tk()
root.title("IMDB MOVIE RECOMMENDER SYSTEM")

mainframe = ttk.Frame(root, padding="3 3 12 12")
mainframe.grid(column=0, row=0, sticky=(N, W, E, S))

filterframe = ttk.Labelframe(mainframe, padding="3 3 12 12", text="Filter")
filterframe.grid(column=0, row=0, sticky=(N, W))
filterframe['borderwidth'] = 2
filterframe['relief'] = 'sunken'


yearfilterframe = ttk.Labelframe(filterframe, padding="3 3 12 12", text="Year")
yearfilterframe.grid(column=0, row=0, sticky=(N, E))
yearfilterframe['borderwidth'] = 2
yearfilterframe['relief'] = 'sunken'
ttk.Label(yearfilterframe, text="MIN").grid(column=0, row=0, sticky="W")
minyear=IntVar()
minyear.set(min_year)
ttk.Entry(yearfilterframe, textvariable=minyear).grid(column=1, row=0, sticky="E")
ttk.Label(yearfilterframe, text="MAX").grid(column=0, row=1, sticky="W")
maxyear=IntVar()
maxyear.set(max_year)
ttk.Entry(yearfilterframe, textvariable=maxyear).grid(column=1, row=1, sticky="E")


numratingfilterframe = ttk.Labelframe(filterframe, padding="3 3 12 12", text="# of Ratings")
numratingfilterframe.grid(column=0, row=1, sticky=(N, E))
numratingfilterframe['borderwidth'] = 2
numratingfilterframe['relief'] = 'sunken'
ttk.Label(numratingfilterframe, text="MIN").grid(column=0, row=0, sticky="E")
minratingcount=IntVar()
minratingcount.set(min_rating_count)
ttk.Entry(numratingfilterframe, textvariable=minratingcount).grid(column=1, row=0, sticky="W")
ttk.Label(numratingfilterframe, text="MAX").grid(column=0, row=1, sticky="E")
maxratingcount=IntVar()
maxratingcount.set(max_rating_count)
ttk.Entry(numratingfilterframe, textvariable=maxratingcount).grid(column=1, row=1, sticky="W")


meanratingfilterframe = ttk.Labelframe(filterframe, padding="3 3 12 12", text="# of Ratings")
meanratingfilterframe.grid(column=0, row=2, sticky=(N, E))
meanratingfilterframe['borderwidth'] = 2
meanratingfilterframe['relief'] = 'sunken'
ttk.Label(meanratingfilterframe, text="MIN").grid(column=0, row=0, sticky="W")
minrating=DoubleVar()
minrating.set(min_mean_rating)
ttk.Entry(meanratingfilterframe, textvariable=minrating).grid(column=1, row=0, sticky="E")
ttk.Label(meanratingfilterframe, text="MAX").grid(column=0, row=1, sticky="W")
maxrating=DoubleVar()
maxrating.set(max_mean_rating)
ttk.Entry(meanratingfilterframe, textvariable=maxrating).grid(column=1, row=1, sticky="E")

phrasefilterframe = ttk.Labelframe(filterframe, padding="3 3 12 12", text="Phrase")
phrasefilterframe.grid(column=0, row=3, sticky="nsew")
phrasefilterframe['borderwidth'] = 2
phrasefilterframe['relief'] = 'sunken'

titlephrase = StringVar()
ttk.Label(phrasefilterframe, text="Title").grid(column=0, row=0, sticky="W")
ttk.Entry(phrasefilterframe, textvariable=titlephrase).grid(column=1, row=0, sticky="nsew")
keywordphrase = StringVar()
ttk.Label(phrasefilterframe, text="Keyword").grid(column=0, row=1, sticky="W")
ttk.Entry(phrasefilterframe, textvariable=keywordphrase).grid(column=1, row=1, sticky="nsew")
genrephrase = StringVar()
ttk.Label(phrasefilterframe, text="Genre").grid(column=0, row=2, sticky="W")
ttk.Entry(phrasefilterframe, textvariable=genrephrase).grid(column=1, row=2, sticky="nsew")

removed_iids = set()

def filter_data(*args):
    remove_iids = set()
    data = data_tree.get_children('')
    #print(data_tree.set(data[0]))    
    for iid in data:
        m = m_[int(iid)]
        # Year
        if m.year < minyear.get():
            remove_iids.add(iid)
        if m.year > maxyear.get():
            remove_iids.add(iid)
        # Rating Count
        if m.num_ratings < minratingcount.get():
            remove_iids.add(iid)
        if m.num_ratings > maxratingcount.get():
            remove_iids.add(iid)
        # MEAN RATING
        if m.mean_rating < minrating.get():
            remove_iids.add(iid)
        if m.mean_rating > maxrating.get():
            remove_iids.add(iid)
        if m.title.find(titlephrase.get()) == -1:
            remove_iids.add(iid)
        
        if genrephrase.get() != "":
            if m.genres == None:
                remove_iids.add(iid)
            else:
                gp = False
                for g in m.genres:
                    if g.find(genrephrase.get()) != -1:
                        gp = True
                        break
                if not gp:
                    remove_iids.add(iid)
        if keywordphrase.get() != "":
            if m.keywords == None:
                remove_iids.add(iid)
            else:
                kp = False
                for k in m.keywords:
                    if k.find(keywordphrase.get()) != -1:
                        kp = True
                        break
                if not kp:
                    remove_iids.add(iid)
        
    #data_tree.delete(list(remove_iids))
    for iid in remove_iids:
        data_tree.delete(iid)
    removed_iids.update(remove_iids)
    #TODO
    # 1. Update the other fields based on the new filtered data
    # 2. There is no way to go back in filtering right now

def reset_filters(*args):
    for iid in removed_iids:
        i = int(iid)
        item = [m_[i].title, m_[i].year, m_[i].num_ratings, m_[i].mean_rating]
        data_tree.insert('', 'end', iid = str(i), values=item)
    sortby(data_tree, 'Title', False)
    removed_iids.clear()
    #TODO Reset the fields to their default values
    
def double_click(e):
    item = data_tree.selection()[0]
    #print("You clicked on", data_tree.set(item))
    #print(m_[int(item)])
    tl = Toplevel(root)
    #ttk.Label(tl, text=data_tree.set(item)).grid(column=0, row=0, sticky="E")
    m = m_[int(item)]
    
    def rate_movie(*args):
        #print(m.title)
        #print(rating.get())
        #ratings[m.title+str(m.year)]=rating.get()
        if rating.get() != "no-opinion":
            ratings[m] = rating.get()
        else:
            ratings.pop(m, None)
        tl.destroy()
    
    ttk.Label(tl, text=m.title + " (" + str(m.year) + ")", font=font.Font(family='Helvetica', size=14, weight='bold')).grid(column=0, row=0, sticky="W")
    if m.genres is not None:
        g = ", ".join(m.genres)
        ttk.Label(tl, text="Genres: " + g).grid(column=0, row=1, sticky="W")      
    if m.keywords is not None:
        #cnames = StringVar(value=countrynames)
        kl= Listbox(tl, listvariable=StringVar(value=m.keywords), height=5)
        kl.grid(column=0, row=2, sticky="nsew")
        vs = ttk.Scrollbar(tl, orient=VERTICAL, command=kl.yview)
        kl.configure(yscrollcommand=vs.set)
        vs.grid(column=1, row=2, sticky="nsew")
        hs = ttk.Scrollbar(tl, orient=HORIZONTAL, command=kl.xview)
        kl.configure(xscrollcommand=hs.set)
        hs.grid(column=0, row=3, sticky="nsew")
    rateframe=ttk.Frame(tl, padding="3 3 12 12")
    rateframe.grid(column=0, row=4, sticky="nsew")        
    rating = StringVar()
    if m in ratings:
        rating.set(ratings[m])
    else:
        rating.set("no-opinion")
    ttk.Radiobutton(rateframe, text='Like', variable=rating, value='like').grid(column=0, row=0, sticky="nsew")
    ttk.Radiobutton(rateframe, text='Dislike', variable=rating, value='dislike').grid(column=1, row=0, sticky="nsew")
    ttk.Radiobutton(rateframe, text='No opinion', variable=rating, value='no-opinion').grid(column=2, row=0, sticky="nsew")
    ttk.Button(rateframe, text="Submit", command=rate_movie).grid(column=1, row=1, sticky = "nsew")
        
    

buttonfilterframe = ttk.Frame(filterframe, padding="3 3 12 12")
buttonfilterframe.grid(column=0, row=4, sticky=(N, E))

ttk.Button(buttonfilterframe, text="Filter", command=filter_data).grid(column=0, row=0, sticky = "W")
ttk.Button(buttonfilterframe, text="Reset", command=reset_filters).grid(column=1, row=0, sticky = "E")

#s = Scale(filterframe, orient=HORIZONTAL, length=200, from_=1900, to=2017 , showvalue=True, label="Minimum Year", command=filter_year)
#s.grid(column=0, row=0, sticky=W)

#s = Scale(filterframe, orient=HORIZONTAL, length=200, from_=0, to=1e3, resolution=1, showvalue=True, label="Minimum # Ratings", command=filter_num_ratings)
#s.grid(column=0, row=1, sticky=W)

#s = Scale(filterframe, orient=HORIZONTAL, length=200, from_=0.0, to=10.0, resolution=0.1, showvalue=True, label="Minimum Mean Rating")
#s.grid(column=0, row=2, sticky=W)




dataframe = ttk.Labelframe(mainframe, padding="3 3 12 12", text="Data")
dataframe.grid(column=1, row=0, sticky="ne")
dataframe['borderwidth'] = 2
dataframe['relief'] = 'sunken'

searchfilterframe = ttk.Frame(dataframe, padding="3 3 12 12")
searchfilterframe['borderwidth'] = 2
searchfilterframe['relief'] = 'sunken'
searchfilterframe.grid(column=0, row=0, sticky="nsew")



# some of the following code is borrowed from https://www.daniweb.com/programming/software-development/threads/350266/creating-table-in-python

data_header = ['Title', 'Year', 'Rating Count', "Mean Rating", 'US Certificate']
#data = [[m.title, m.year, m.num_ratings, m.mean_rating] for m in m_]

data_tree = ttk.Treeview(dataframe, columns=data_header, show="headings", selectmode='browse')

vsb = ttk.Scrollbar(dataframe, orient="vertical", command=data_tree.yview)
hsb = ttk.Scrollbar(dataframe, orient="horizontal", command=data_tree.xview)

data_tree.configure(yscrollcommand=vsb.set, xscrollcommand=hsb.set)
data_tree.grid(column=0, row=1, sticky='nsew')

data_tree.bind("<Double-1>", double_click)

vsb.grid(column=1, row=1, sticky='ns')
hsb.grid(column=0, row=2, sticky='ew')

dataframe.grid_columnconfigure(0, weight=1)
dataframe.grid_rowconfigure(0, weight=1)


for col in data_header:
    data_tree.heading(col, text=col.title(), command=lambda c=col: sortby(data_tree, c, 0))
    # adjust the column's width to the header string
    data_tree.column(col, width=tkFont.Font().measure(col.title()))
    
for i in range(len(m_)):
    item = [m_[i].title, m_[i].year, m_[i].num_ratings, m_[i].mean_rating, m_[i].certificate]
    data_tree.insert('', 'end', iid = str(i), values=item)
    # adjust column's width if necessary to fit each value
    for ix, val in enumerate(item):
        col_w = tkFont.Font().measure(val)
        if data_tree.column(data_header[ix],width=None)<col_w:
            data_tree.column(data_header[ix], width=col_w)

root.mainloop()

In [83]:
ratings = {}

look_for_genres = set()
look_for_genres.update(['Documentary'])

look_for_keywords = set()
look_for_keywords.update(['nature'])

certificates = set()
certificates.update(["G", "PG", "TV-G", "TV-PG"])


for m in m_:
    if m.certificate in certificates:
        if m.genres is not None:
            mgs = set()
            mgs.update(m.genres)
            if len(look_for_genres.intersection(mgs)) == len(look_for_genres):
                #ratings[m] = 'like'
                if m.keywords is not None:
                    mks = set()
                    mks.update(m.keywords)
                    if len(look_for_keywords.intersection(mks)) == len(look_for_keywords):
                        ratings[m] = 'like'

In [84]:
len(ratings)

25

In [85]:
#for m in ratings:
    #print(m)
    #print(m.genres)
    #print()

In [86]:
#pickle.dump(ratings, open( "kidsratings.p", "wb" ))

In [87]:
# Vectorize
# #Ratings, z-scored Mean Rating, #Ratings x z-scored meanrating, Genres, keywords
Genres = set()
Keywords = set()
for m in ratings:
    if m.genres is not None:
        for g in m.genres:
            Genres.add(g)
    if m.keywords is not None:
        for k in m.keywords:
            Keywords.add(k)

In [88]:
len(Genres)

9

In [89]:
len(Keywords)

695

In [90]:
Genres = list(Genres)

In [91]:
Keywords = list(Keywords)

In [92]:
import numpy as np

In [93]:
r = []
for m in movies:
    r.append(m.mean_rating)

In [94]:
r_mean = np.mean(r)
r_var = np.var(r)

In [95]:
from scipy.sparse import csr_matrix, vstack

In [96]:
D = []
y = []
for m in m_:
    #if m not in ratings:
    #    continue
    d = []
    #d.append(m.num_ratings)
    #d.append((m.mean_rating - r_mean)/r_var)
    #d.append(d[0]*d[1])
    m_g = set()
    if m.genres is not None:
        m_g.update(m.genres)
    m_k = set()
    if m.keywords is not None:
        m_k.update(m.keywords)
    for g in Genres:
        if g in m_g:
            d.append(1.)
        else:
            d.append(0.)
    for k in Keywords: # VERY INEFFICIENT. FIX!
        if k in m_k:
            d.append(1.)
        else:
            d.append(0.)
    D.append(csr_matrix(d))
    if m in ratings:
        y.append(1)
    else:
        y.append(0)

In [97]:
len(D)

30587

In [98]:
d = D[0]

In [99]:
for d_ in D[1:]:
    d = vstack([d, d_])

In [100]:
d.shape

(30587, 704)

In [101]:
len(y)

30587

In [102]:
y = np.array(y)

In [103]:
d[np.where(y==1)]

<25x704 sparse matrix of type '<class 'numpy.float64'>'
	with 957 stored elements in Compressed Sparse Row format>

In [104]:
from sklearn import svm
from sklearn.preprocessing import normalize

In [105]:
d = normalize(d)

In [106]:
clf = svm.OneClassSVM(kernel='linear')

In [107]:
clf.fit(d[np.where(y==1)])

OneClassSVM(cache_size=200, coef0=0.0, degree=3, gamma='auto',
      kernel='linear', max_iter=-1, nu=0.5, random_state=None,
      shrinking=True, tol=0.001, verbose=False)

In [108]:
clf.coef_

<1x704 sparse matrix of type '<class 'numpy.float64'>'
	with 695 stored elements in Compressed Sparse Row format>

In [109]:
indices = np.argsort(np.abs(clf.coef_.toarray()[0]))[::-1]

In [110]:
for i in indices[:100]:
    print(clf.coef_.toarray()[0][i])

2.01479351869
2.01479351869
0.502038351525
0.492041148973
0.463418885484
0.461206965958
0.441293499162
0.389214893561
0.382033747798
0.376611201898
0.373989033135
0.371195533367
0.366094744509
0.361077440554
0.358154250821
0.352931584756
0.332718736208
0.329244108495
0.32313140022
0.302215187847
0.299053328095
0.293704264368
0.291614222049
0.281165319843
0.279354282467
0.279138071577
0.266096789031
0.264823059705
0.264823059705
0.264823059705
0.253744711798
0.253744711798
0.253744711798
0.251236999096
0.251236999096
0.251036021606
0.244941215686
0.240581552044
0.240581552044
0.240581552044
0.240542374475
0.239946326095
0.238328265159
0.237186646706
0.237186646706
0.230478375656
0.230012034283
0.230012034283
0.230012034283
0.230012034283
0.230012034283
0.230012034283
0.230012034283
0.22766288454
0.227517718778
0.226783166341
0.225825653776
0.225825653776
0.224582365192
0.224278398376
0.224278398376
0.215867563016
0.215867563016
0.214075409895
0.213392412383
0.210954113168
0.206399612578

In [111]:
for i in indices[:30]:
    if i < len(Genres):
        print(Genres[i])
    else:
        print(Keywords[i-len(Genres)])

nature
Documentary
voice-over-narration
water
wildlife
nature-documentary
africa
narration
coral-reef
river
underwater
penguin
migration
survival
physicist
Short
island
ice
bird
one-word-title
tv-mini-series
human-animal-relationship
science
moon
elephant
ocean
scientist
slow-motion
environment
time-lapse-photography


In [112]:
np.where(y==1)

(array([ 1482,  2389,  2390,  3115,  3350,  3677,  3976,  3977,  3978,
         5095,  5582,  6395,  7176,  8136, 10619, 11901, 15893, 16330,
        17927, 18201, 20488, 21374, 26146, 27867, 28918], dtype=int64),)

In [113]:
positive = np.where(clf.predict(d) != -1)[0]

In [114]:
len(positive)

281

In [115]:
df = (clf.decision_function(d)).flatten()

In [116]:
indices = np.argsort(df)[::-1]

In [117]:
for i in indices[:20]:
    print(m_[i])
    #print(df[i])
    #print(m_[i].keywords)
    #print()

"The Trials of Life"	(1990)	(Unk)
"The Life of Mammals"	(2002)	(Unk)
"Alone"	(2015)	(Unk)
"Africa"	(2013)	(Unk)
"The National Parks: America's Best Idea"	(2009)	(Unk)
"Life on Earth"	(1979)	(Unk)
If a Tree Falls: A Story of the Earth Liberation Front	(2011)	(Unrated)
"Planet Earth II"	(2016)	(TV-G)
Clinton Cash	(2016)	(Unk)
Startup.com	(2001)	(R)
"Life in the Freezer"	(1993)	(Unk)
Hungry for Change	(2012)	(Unk)
Hurricane of Fun: The Making of Wet Hot	(2015)	(Not Rated)
The September Issue	(2009)	(PG-13)
Once in a Lifetime: The Extraordinary Story of the New York Cosmos	(2006)	(PG-13)
Please Vote for Me	(2007)	(Unk)
Man vs Snake: The Long and Twisted Tale of Nibbler	(2015)	(Unk)
Requiem for the American Dream	(2015)	(Unk)
We Are Legion: The Story of the Hacktivists	(2012)	(Unk)
More Than a Game	(2008)	(PG)


In [118]:
df[indices[:20]]

array([ 1.13370742,  0.87839728,  0.81931042,  0.6237964 ,  0.51604214,
        0.49007649,  0.39612442,  0.36502607,  0.29915262,  0.29915262,
        0.29915262,  0.29915262,  0.29915262,  0.29915262,  0.29915262,
        0.29915262,  0.29915262,  0.29915262,  0.29915262,  0.29915262])